# Providing Data for Plots and Tables
https://bokeh.pydata.org/en/latest/docs/user_guide/data.html

In [2]:
%pylab inline
import bokeh
from bokeh.plotting import figure, show #, output_file
from bokeh.io import output_notebook

output_notebook()

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [3]:
p = figure(width=360, height=360)

### Providing data directly

In [7]:
x_values = [1, 2, 3, 4, 5]
y_values = [6, 7, 2, 3, 6]

p = figure(width=360, height=360)
p.circle(x=x_values, y=y_values)
show(p)

### ColumnDataSource

In [8]:
from bokeh.models import ColumnDataSource

data = {'x_values': [1, 2, 3, 4, 5],
        'y_values': [6, 7, 2, 3, 6]}

source = ColumnDataSource(data=data)

p = figure(width=360, height=360)
p.circle(x='x_values', y='y_values', source=source)
show(p)

### Streaming
ColumnDataSource streaming is an efficient way to append new data to a CDS. By using the stream method, Bokeh **only sends new data to the browser instead of the entire dataset**. The stream method takes a new_data parameter containing a dict mapping column names to sequences of data to be appended to the respective columns. It additionally takes an optional argument rollover, which is the maximum length of data to keep (data from the beginning of the column will be discarded). The default rollover value of None allows data to grow unbounded.

In [9]:
source = ColumnDataSource(data=dict(foo=[], bar=[]))

# has new, identical-length updates for all columns in source
new_data = {
    'foo' : [10, 20],
    'bar' : [100, 200],
}

source.stream(new_data)

p = figure(width=360, height=360)
p.circle(x='foo', y='bar', source=source)
show(p)

### Filtering data with CDSView

```
from bokeh.models import ColumnDataSource, CDSView

source = ColumnDataSource(some_data)
view = CDSView(source=source, filters=[filter1, filter2])

p = figure()
p.circle(x="x", y="y", source=source, view=view)
```

### IndexFilter

In [10]:
from bokeh.models import ColumnDataSource, CDSView, IndexFilter
from bokeh.layouts import gridplot 

source = ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[1, 2, 3, 4, 5]))

view = CDSView(source=source, 
               filters=[IndexFilter([0, 2, 4])])

tools = ["box_select", "hover", "reset"]
p = figure(plot_height=300, plot_width=300, tools=tools)
p.circle(x="x", y="y", size=10, hover_color="red", source=source)

p_filtered = figure(plot_height=300, plot_width=300, tools=tools)
p_filtered.circle(x="x", y="y", size=10, hover_color="red", source=source,
                  view=view)

show(gridplot([[p, p_filtered]]))

### BooleanFilter

In [11]:
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
from bokeh.layouts import gridplot 

source = ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[1, 2, 3, 4, 5]))

booleans = [True if y_val > 2 else False for y_val in source.data['y']]

view = CDSView(source=source, 
               filters=[BooleanFilter(booleans)])

tools = ["box_select", "hover", "reset"]
p = figure(plot_height=300, plot_width=300, tools=tools)
p.circle(x="x", y="y", size=10, hover_color="red", source=source)

p_filtered = figure(plot_height=300, plot_width=300, tools=tools,
                    x_range=p.x_range, y_range=p.y_range)
p_filtered.circle(x="x", y="y", size=10, hover_color="red", source=source,
                  view=view)

show(gridplot([[p, p_filtered]]))

### GroupFilter

In [12]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter

from bokeh.sampledata.iris import flowers

source = ColumnDataSource(flowers)

view1 = CDSView(source=source,
                filters=[GroupFilter(column_name='species', group='versicolor')])

plot_size_and_tools = {'plot_height': 300, 'plot_width': 300,
                        'tools':['box_select', 'reset', 'help']}

p1 = figure(title="Full data set", **plot_size_and_tools)
p1.circle(x='petal_length', y='petal_width', source=source, color='black')

p2 = figure(title="Setosa only", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools)
p2.circle(x='petal_length', y='petal_width', source=source, color='red',
          view=view1)

show(gridplot([[p1, p2]]))

### Linked selection

In [13]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource 

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0',
            source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1',
             source=source)

p = gridplot([[left, right]])

show(p)

### Linked selection with filtered data

In [14]:
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

# create a view of the source for one plot to use
view = CDSView(source=source,
               filters=[BooleanFilter([True if y > 250 or y < 100 else False for y in y1])])

TOOLS = "box_select,lasso_select,hover,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', size=10, hover_color="firebrick",
            source=source)

# create another new plot, add a renderer that uses the view of the data source
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', size=10, hover_color="firebrick", 
             source=source,
             view=view)

p = gridplot([[left, right]])

show(p)